# Пример агрегации рассчитанных сервисов

Этот ноутбук показывает, как использовать `MigrationFlowModel` для объединения уже подсчитанных показателей обеспеченности по отдельным сервисам в итоговый JSON по городам.

## Требуемые входные данные

1. Файл c перечнем городов: столбцы `city_id`, `city_name`, `is_anchor` (и при наличии `population`).
2. Набор файлов (pickle/parquet/csv) для каждого сервиса, содержащих столбцы `demand`, `demand_within`, `demand_without`, `capacity`, `capacity_left`.

Ниже оставлены заглушки путей — замените их на реальные значения перед запуском.

In [ ]:
%load_ext reload
%reload 2

In [1]:
from pathlib import Path
import json

from townsnet.provision.model import MigrationFlowModel

In [2]:
# TODO: укажите реальные пути к файлам
CITY_INFO_PATH = Path("data/townsnet10/towns.geojson")
SERVICE_FILES = {
    "ООПТ": Path("data/townsnet10/ООПТ.geojson"),
}
OUTPUT_JSON = Path("data/townsnet10/city_profiles.json")

In [4]:
model = MigrationFlowModel()

if not CITY_INFO_PATH.exists():
    raise FileNotFoundError(
        "Не найден файл с информацией о городах. Обновите переменную CITY_INFO_PATH перед запуском."
    )
model.load_city_info(CITY_INFO_PATH)

# missing_services = [name for name, path in SERVICE_FILES.items() if not path.exists()]
# if missing_services:
#     raise FileNotFoundError(
#         "Не найдены файлы для сервисов: " + ", ".join(missing_services) + ". Обновите словарь SERVICE_FILES."
#     )

model.load_service_results(SERVICE_FILES)
profiles = model.build_profiles()

print(f"Сформировано профилей: {len(profiles)}")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Просмотр одного профиля (замените 0 на нужный city_id)
sample_city_id = next(iter(profiles))
print(json.dumps({sample_city_id: profiles[sample_city_id]}, ensure_ascii=False, indent=2))

In [ ]:
OUTPUT_JSON.parent.mkdir(parents=True, exist_ok=True)
model.save_city_json(OUTPUT_JSON, by="id")
print(f"Результат сохранён в {OUTPUT_JSON}")